<a href="https://colab.research.google.com/github/cwattsnogueira/rating-predictor-spam-detection-review-summarizer/blob/main/ReviewModerationAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Upload your ZIP file (bert_oversampling_model.zip)
from google.colab import files
uploaded = files.upload()

# Step 2: Extract filename
import os
zip_filename = next(iter(uploaded))  # gets the first uploaded file

# Step 3: Create target folder
target_folder = "/content/"
os.makedirs(target_folder, exist_ok=True)

# Step 4: Unzip into the folder
import zipfile
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

print(f" Unzipped '{zip_filename}' into '{target_folder}'")

Saving bert_oversampling_model.zip to bert_oversampling_model.zip
 Unzipped 'bert_oversampling_model.zip' into '/content/'


In [2]:
# Colab cell 1: install dependencies (run once)
!pip install -q transformers gradio torch pandas scikit-learn joblib textblob tqdm
# optional: if TextBlob needs corpora it may warn; for safety we will fall back gracefully if missing.

In [3]:
# Colab cell 2: main app
import os
import io
import re
import math
import json
import torch
import joblib
import pandas as pd
import numpy as np
from textblob import TextBlob
from transformers import BertTokenizer, BertForSequenceClassification
import gradio as gr
from torch.nn.functional import softmax
from tqdm.auto import tqdm

In [4]:
# ------------- CONFIG -------------
MODEL_DIR = "/content/bert_oversampling_model"  # update if needed
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32

In [5]:
# ------------- HELPERS -------------
def safe_read_csv(uploaded_file):
    """
    Robust CSV reader that handles Gradio uploads, paths, or file-like objects.
    """
    try:
        if hasattr(uploaded_file, "name"):
            return pd.read_csv(uploaded_file.name, encoding='utf-8', engine='python', on_bad_lines='skip')
        if isinstance(uploaded_file, str) and os.path.exists(uploaded_file):
            return pd.read_csv(uploaded_file, encoding='utf-8', engine='python', on_bad_lines='skip')
        if hasattr(uploaded_file, "read"):
            content = uploaded_file.read()
            if isinstance(content, bytes):
                try:
                    s = content.decode('utf-8')
                except UnicodeDecodeError:
                    s = content.decode('latin1')
            else:
                s = content
            return pd.read_csv(io.StringIO(s), engine='python', on_bad_lines='skip')
    except Exception as e:
        raise RuntimeError(f"Failed to read CSV: {e}")
    raise RuntimeError("Unsupported file type for CSV input.")

In [6]:
# Column name mapping heuristics
PRODUCT_CANDIDATES = ["product_name", "product", "name", "productName"]
TITLE_CANDIDATES = ["review_title", "title", "reviewTitle"]
TEXT_CANDIDATES = ["review_text", "review", "text", "reviews.text", "review_text_clean", "clean_text"]

def map_columns(df):
    cols = df.columns.str.lower()
    col_map = {c.lower(): c for c in df.columns}

    def choose(candidates):
        for c in candidates:
            if c in col_map:
                return col_map[c]
        return None

    product_col = choose(PRODUCT_CANDIDATES)
    title_col = choose(TITLE_CANDIDATES)
    text_col = choose(TEXT_CANDIDATES)

    if text_col is None:
        lengths = {}
        for c in df.columns:
            try:
                sample = df[c].dropna().astype(str).head(100)
                avg_len = sample.str.split().apply(len).mean()
                lengths[c] = avg_len
            except Exception:
                lengths[c] = 0
        candidate = max(lengths, key=lambda k: lengths[k])
        if lengths[candidate] > 3:
            text_col = candidate

    return product_col, title_col, text_col

In [7]:
# Heuristics for explainability
URL_RE = re.compile(r'https?://\S+')
EMAIL_RE = re.compile(r'\S+@\S+')
PHONE_RE = re.compile(r'(\+?\d[\d\-\s]{6,}\d)')

In [8]:
CATEGORY_KEYWORDS = {
    'skincare': ['cream','moisturizer','soap','lotion','cleanser'],
    'haircare': ['shampoo','conditioner','hair'],
    'makeup': ['lipstick','foundation','concealer','makeup'],
    'fragrance': ['perfume','scent','aroma','fragrance'],
    'food': ['taste','sauce','cake','pizza','ketchup','flavor'],
    'household': ['detergent','cleaner','glass','spray'],
    'home': ['lamp','bed','furniture','shelf','blanket'],
    'entertainment': ['movie','dvd','series','album']
}

In [9]:
def compute_heuristics(product, title, review_text):
    text = ("" if pd.isna(review_text) else str(review_text)).strip()
    product = ("" if pd.isna(product) else str(product)).strip().lower()

    tokens = re.findall(r'\w+', text.lower())
    token_count = len(tokens)

    repetition_score = 0.0
    if token_count:
        vc = pd.Series(tokens).value_counts(normalize=True)
        repetition_score = float(vc.max())

    product_overlap = 0
    if product:
        pname_tokens = set(re.findall(r'\w+', product))
        product_overlap = int(len(pname_tokens & set(tokens)) > 0)

    has_url = int(bool(URL_RE.search(text)))
    has_email = int(bool(EMAIL_RE.search(text)))
    has_phone = int(bool(PHONE_RE.search(text)))

    try:
        polarity = TextBlob(text).sentiment.polarity
    except Exception:
        polarity = 0.0

    short_extreme_flag = int((token_count < 6) and (abs(polarity) > 0.8))

    unrelated_flag = 0
    for cat, keywords in CATEGORY_KEYWORDS.items():
        for kw in keywords:
            if re.search(r'\b' + re.escape(kw) + r'\b', text.lower()):
                if product and kw not in product:
                    unrelated_flag = 1
                elif not product:
                    unrelated_flag = 1

    return {
        "token_count": int(token_count),
        "repetition_score": repetition_score,
        "product_overlap": int(product_overlap),
        "has_url": has_url,
        "has_email": has_email,
        "has_phone": has_phone,
        "polarity": float(polarity),
        "short_extreme_flag": short_extreme_flag,
        "unrelated_flag": unrelated_flag
    }

In [10]:
# -------------- MODEL LOADING --------------
print("Loading model...")
if not os.path.isdir(MODEL_DIR):
    raise RuntimeError(f"Model directory {MODEL_DIR} not found.")
tokenizer = BertTokenizer.from_pretrained(MODEL_DIR)
model = BertForSequenceClassification.from_pretrained(MODEL_DIR)
model.to(DEVICE)
model.eval()
print("Model loaded on", DEVICE)

Loading model...
Model loaded on cpu


In [11]:
# -------------- PREDICTION --------------
def batch_predict_texts(texts, batch_size=BATCH_SIZE):
    preds, probs = [], []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
            inputs = {k:v.to(DEVICE) for k,v in inputs.items()}
            outputs = model(**inputs)
            logits = outputs.logits
            batch_probs = softmax(logits, dim=-1).cpu().numpy()
            batch_preds = np.argmax(batch_probs, axis=1)
            preds.extend(batch_preds.tolist())
            probs.extend(batch_probs.tolist())
    return preds, probs

In [12]:
# ---------------- CLASSIFICATION FUNCTIONS ----------------
def classify_review(product, title, review, reviewer=None, date=None):
    import re
    from textblob import TextBlob

    # Combine title and review
    text = f"{title.strip()}. {review.strip()}" if title else review.strip()

    # Heuristics
    tokens = text.split()
    token_count = len(tokens)
    repetition_score = max([tokens.count(t) for t in set(tokens)]) / token_count if token_count > 0 else 0
    product_overlap = 1 if product.lower() in text.lower() else 0
    polarity = TextBlob(text).sentiment.polarity
    short_extreme_flag = token_count < 12 and polarity > 0.7
    unrelated_flag = any(kw in text.lower() for kw in ["buy now", "click here", "visit site"])
    has_url = bool(re.search(r"http[s]?://", text))
    has_email = bool(re.search(r"\S+@\S+", text))
    has_phone = bool(re.search(r"\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b", text))

    heur = {
        "token_count": token_count,
        "repetition_score": repetition_score,
        "product_overlap": product_overlap,
        "polarity": polarity,
        "short_extreme_flag": short_extreme_flag,
        "unrelated_flag": unrelated_flag,
        "has_url": has_url,
        "has_email": has_email,
        "has_phone": has_phone
    }

    # Model prediction
    preds, probs = batch_predict_texts([text])
    prob = probs[0]
    fake_score = float(prob[1])
    genuine_score = float(prob[0])

    # Heuristic post-processing (recalibrated)
    if polarity < -0.15 and "refund" in text.lower():
        fake_score -= 0.2  # rescue negative reviews with refund
    if token_count < 12 and polarity > 0.5:
        fake_score += 0.15  # short + strongly positive → suspicious
    if product_overlap and repetition_score > 0.2:
        fake_score += 0.2  # repeated product name + promo
    if token_count > 25 and polarity < 0.3 and repetition_score < 0.1:
        fake_score -= 0.1  # long, neutral, low repetition → likely genuine

    # Normalize
    fake_score = max(0, min(1, fake_score))
    genuine_score = 1 - fake_score
    pred = 1 if fake_score > 0.6 else 0

    label = "⚠️ Fake" if pred == 1 else "✅ Genuine"
    suggestion = "This review may contain vague or promotional language. Manual check advised." if pred == 1 else "This review appears authentic with specific details."

    explanation_lines = [
        f"- Token count: {heur['token_count']}",
        f"- Repetition score: {heur['repetition_score']:.2f}",
        f"- Product name overlap: {heur['product_overlap']}",
        f"- Sentiment polarity (TextBlob): {heur['polarity']:.2f}",
        f"- Short & extreme sentiment: {bool(heur['short_extreme_flag'])}",
        f"- Unrelated keyword flag: {heur['unrelated_flag']}"
    ]
    if has_url or has_email or has_phone:
        explanation_lines.append(f"- Contains URL/email/phone: url={has_url}, email={has_email}, phone={has_phone}")

    explanation = "\n".join(explanation_lines)

    out_md = f"""**Prediction:** {label}
**Confidence:** {fake_score*100:.2f}% if Fake, {genuine_score*100:.2f}% if Genuine

**Class 0 (Genuine):** {genuine_score:.2%}
**Class 1 (Fake):** {fake_score:.2%}

**Suggestion:** {suggestion}

**Heuristics / Explanation:**
{explanation}
"""
    return out_md

In [13]:
def batch_classify(uploaded_file):
    try:
        df = safe_read_csv(uploaded_file)
    except Exception as e:
        return f"❌ Error reading CSV: {e}"

    if df.shape[0] == 0:
        return "❌ CSV is empty."

    product_col, title_col, text_col = map_columns(df)
    if text_col is None:
        return "❌ Could not find a review text column."

    df['__product'] = df[product_col] if product_col in df.columns else ""
    df['__title'] = df[title_col] if title_col in df.columns else ""
    df['__text'] = df[text_col]

    texts = (df['__title'].fillna('').astype(str).str.strip() + ". " + df['__text'].fillna('').astype(str)).tolist()
    preds, probs = batch_predict_texts(texts)

    results = []
    for i, row in df.reset_index(drop=True).iterrows():
        text = f"{row['__title']}. {row['__text']}"
        prob = probs[i]
        fake_score = float(prob[1])
        genuine_score = float(prob[0])

        # Heuristics
        tokens = text.split()
        token_count = len(tokens)
        repetition_score = max([tokens.count(t) for t in set(tokens)]) / token_count if token_count > 0 else 0
        product_overlap = 1 if row['__product'].lower() in text.lower() else 0
        polarity = TextBlob(text).sentiment.polarity

        # Heuristic post-processing (recalibrated)
        if polarity < -0.15 and "refund" in text.lower():
            fake_score -= 0.2
        if token_count < 12 and polarity > 0.5:
            fake_score += 0.15
        if product_overlap and repetition_score > 0.2:
            fake_score += 0.2
        if token_count > 25 and polarity < 0.3 and repetition_score < 0.1:
            fake_score -= 0.1

        fake_score = max(0, min(1, fake_score))
        genuine_score = 1 - fake_score
        pred = 1 if fake_score > 0.6 else 0

        label = "Fake" if pred == 1 else "Genuine"
        suggestion = "May contain vague or promotional language." if pred == 1 else "Likely authentic."
        explanation = f"tokens={token_count}, repetition={repetition_score:.2f}, product_overlap={product_overlap}, polarity={polarity:.2f}"

        results.append({
            "product_name": row['__product'],
            "review_title": row['__title'],
            "review_text": row['__text'],
            "prediction": label,
            "confidence": f"{fake_score*100:.2f}%" if pred == 1 else f"{genuine_score*100:.2f}%",
            "class_0_genuine": f"{genuine_score:.2%}",
            "class_1_fake": f"{fake_score:.2%}",
            "suggestion": suggestion,
            "explanation": explanation
        })

    return pd.DataFrame(results)

In [14]:
# -------------- GRADIO UI --------------
with gr.Blocks(title="Review Moderation Assistant") as demo:
    gr.Markdown("##  Review Moderation Assistant\nUpload a CSV or evaluate a single review. The model returns a prediction, confidence, and rule-based explanation flags.")
    with gr.Tab("Single Review"):
        product = gr.Textbox(label="Product Name", placeholder="e.g. Acme Face Cream")
        title = gr.Textbox(label="Review Title", placeholder="Optional short title")
        review = gr.Textbox(label="Review Text", lines=6, placeholder="Paste review text here")
        reviewer = gr.Textbox(label="Reviewer (optional)")
        date = gr.Textbox(label="Review Date (optional)")
        eval_btn = gr.Button("Evaluate Review")
        out_md = gr.Markdown()
        eval_btn.click(fn=classify_review, inputs=[product, title, review, reviewer, date], outputs=out_md)

    with gr.Tab("Batch Review (CSV)"):
        gr.Markdown("CSV must contain a review text column. The app will try to map common column names: `review_text`, `review`, `text`.")
        file_input = gr.File(label="Upload CSV", file_types=[".csv", ".txt"])
        run_btn = gr.Button("Run CSV Classification")
        batch_output = gr.DataFrame()
        run_btn.click(fn=batch_classify, inputs=file_input, outputs=batch_output)

    gr.Markdown("### Notes\n- Place your trained model in `/content/bert_oversampling_model`.\n- Example CSV columns: `product_name`, `review_title`, `review_text`")

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://885978abb8416d6513.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
